<center>
    <img src="https://gitlab.com/ibm/skills-network/courses/placeholder101/-/raw/master/labs/module%201/images/IDSNlogo.png"  width="300" alt="cognitiveclass.ai logo"  />
</center>

<h1 align=center><font size = 5>Assignment: SQL Notebook for Peer Assignment</font></h1>

Estimated time needed: **60** minutes.

## Introduction

Using this Python notebook you will:

1.  Understand the Spacex DataSet
2.  Load the dataset  into the corresponding table in a Db2 database
3.  Execute SQL queries to answer assignment questions


## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones.

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage.

Therefore if we can determine if the first stage will land, we can determine the cost of a launch.

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.


### Download the datasets

This assignment requires you to load the spacex dataset.

In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. Click on the link below to download and save the dataset (.CSV file):

<a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01" target="_blank">Spacex DataSet</a>


In [1]:
#Crear tabla
def create_table(Dataframe, table_name):
  query = "create table {} (".format(table_name)
  n=0
  for name in Dataframe.columns:
    tipo = type(Dataframe[name][0])
    if tipo == np.int64:
      sql_tipo = "int(255)"
    elif tipo is datetime.date:
      sql_tipo = "date"
    elif tipo is datetime.time:
      sql_tipo = "time"
    elif tipo == "np.float64":
      sql_tipo = "float(10,5)"
    elif tipo == "bool":
      sql_tipo = "boolea"
    else: 
      sql_tipo = "varchar(50)"
    if n == len(Dataframe.columns)-1:
      query += name + ' ' + sql_tipo + ')'
      break
    query += name + ' ' + sql_tipo + ', '
    n += 1
  return query

In [2]:
#Funcion para crear query de carga de datos
def query_insert_data(table_name, Dataframe):
  query = "insert into " + table_name + " ("
  n=1
  for name in Dataframe.columns:
    if n==len(Dataframe.columns):
      query += name + ")"
      break
    query += name + ", "
    n += 1

  query += " values "
  lista = Dataframe.values.tolist()
  p=0
  for row in lista:
    query += "("
    m=0
    for value in row:
      if type(value) != int or type(value) != float:
        value = "'" + str(value) + "'"

      value = str(value)
      if p == len(lista)-1 and m == len(row)-1:
        query += value + ");"
        break

      if m == len(row)-1:
        query += value + "), "
        break

      query += value + ", "
      m += 1
    p += 1
  return query



In [3]:
#Query executer
def query_executer(query,cnx):
  df = pd.read_sql_query(query, cnx)
  return df

In [8]:
#Query executer
def query_executer(query,cnx):
  df = pd.read_sql_query(query, cnx)
  return df

### Connect to the database

Let us first load the SQL extension and establish a connection with the database


In [4]:
!pip install pymysql

In [5]:
import pymysql
import sqlalchemy
import pandas as pd
import numpy as np
import datetime

In [6]:
user = "admin"
host = 'basespacey.camxcm5aw0vw.us-west-1.rds.amazonaws.com'
password = 'b4sur4098'

In [206]:
cnx = pymysql.connect(host=host,
                      user=user,
                      password=password)
cursor = cnx.cursor()

cursor.execute('SHOW DATABASES')
for (databases) in cursor:
  print(databases[0])

basespacey
information_schema
mysql
performance_schema
sys


In [13]:
#si no existe
cursor.execute("create database basespacey")

1

In [144]:
db_name = "basespacey"
table_name = "spacey"

In [207]:
cursor.execute("use {}".format(db_name))

0

In [24]:
from google.colab import files
uploaded = files.upload()

Saving Spacex.csv to Spacex.csv


In [10]:
data = pd.read_csv('Spacex.csv')
df = pd.DataFrame(data)

In [11]:
dates = [datetime.datetime.strptime(e, '%d-%m-%Y').date() for e in df['Date']]
time = [datetime.datetime.strptime(i, '%H:%M:%S').time() for i in df['Time (UTC)']]

df['Date'] = dates
df['Time (UTC)'] = time

df.rename(columns={"Time (UTC)":"Time", "Landing _Outcome":"Landing_Outcome"}, inplace=True)

In [16]:
cursor.execute(create_table(df, table_name))

create table spacey (Date date, Time time, Booster_Version varchar(50), Launch_Site varchar(50), Payload varchar(50), PAYLOAD_MASS__KG_ int(255), Orbit varchar(50), Customer varchar(50), Mission_Outcome varchar(50), Landing_Outcome varchar(50))


0

In [208]:
cursor.execute("show tables")
result = cursor.fetchall()
for data in result:
  print(data)

('spacey',)


In [148]:
query_executer("SELECT column_name FROM INFORMATION_SCHEMA. COLUMNS WHERE TABLE_NAME = '{}' ".format(table_name), cnx)

,COLUMN_NAME
0,Booster_Version
1,Customer
2,Date
3,Landing_Outcome
4,Launch_Site
5,Mission_Outcome
6,Orbit
7,Payload
8,PAYLOAD_MASS__KG_
9,Time


In [150]:
cursor.execute(query_insert_data(table_name, df))
cnx.commit()

In [151]:
#test
query_executer("SELECT Customer, Date from {} limit 10".format(table_name), cnx)

,Customer,Date
0,SpaceX,2010-06-04
1,NASA (COTS) NRO,2010-12-08
2,NASA (COTS),2012-05-22
3,NASA (CRS),2012-10-08
4,NASA (CRS),2013-03-01
5,MDA,2013-09-29
6,SES,2013-12-03
7,Thaicom,2014-01-06
8,NASA (CRS),2014-04-18
9,Orbcomm,2014-07-14


In [152]:
query_executer("select count(*) from {}".format(table_name), cnx)

,count(*)
0,101


In [203]:
cursor.close()

In [204]:
cnx.close()

## Tasks

Now write and execute SQL queries to solve the assignment tasks.

### Task 1

##### Display the names of the unique launch sites  in the space mission


In [153]:
query_executer("SELECT DISTINCT Launch_Site FROM {}".format(table_name),cnx)

,Launch_Site
0,CCAFS LC-40
1,VAFB SLC-4E
2,KSC LC-39A
3,CCAFS SLC-40


### Task 2

##### Display 5 records where launch sites begin with the string 'CCA'


In [210]:
query_executer("SELECT distinct Launch_Site from {}".format(table_name), cnx)

,Launch_Site
0,CCAFS LC-40
1,VAFB SLC-4E
2,KSC LC-39A
3,CCAFS SLC-40


### Task 3

##### Display the total payload mass carried by boosters launched by NASA (CRS)


In [155]:
query_executer("select sum(PAYLOAD_MASS__KG_) from {} where Customer = 'NASA (CRS)'".format(table_name) ,cnx)

,sum(PAYLOAD_MASS__KG_)
0,45596.0


### Task 4

##### Display average payload mass carried by booster version F9 v1.1


In [156]:
query_executer("select avg(PAYLOAD_MASS__KG_) from {} where Booster_Version like 'F9 v1.1%'".format(table_name) ,cnx)

,avg(PAYLOAD_MASS__KG_)
0,2534.6667


### Task 5

##### List the date when the first successful landing outcome in ground pad was acheived.

*Hint:Use min function*


In [157]:
query_executer("select min(Date) from {} where Landing_Outcome = 'Success (ground pad)'".format(table_name),cnx)

,min(Date)
0,2015-12-22


In [158]:
query_executer("select distinct Landing_Outcome from {}".format(table_name), cnx)

,Landing_Outcome
0,Failure (parachute)
1,No attempt
2,Uncontrolled (ocean)
3,Controlled (ocean)
4,Failure (drone ship)
5,Precluded (drone ship)
6,Success (ground pad)
7,Success (drone ship)
8,Success
9,Failure


### Task 6

##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [159]:
query_executer("select Booster_Version from {} where Landing_Outcome = 'Success (drone ship)' and PAYLOAD_MASS__KG_>4000 and PAYLOAD_MASS__KG_<6000;".format(table_name),cnx)

,Booster_Version
0,F9 FT B1022
1,F9 FT B1026
2,F9 FT B1021.2
3,F9 FT B1031.2


### Task 7

##### List the total number of successful and failure mission outcomes


In [160]:
query_executer("select count(Mission_Outcome) from {} where Mission_Outcome = 'Success' or Mission_Outcome = 'Failure (in Flight)'".format(table_name),cnx)

,count(Mission_Outcome)
0,99


### Task 8

##### List the   names of the booster_versions which have carried the maximum payload mass. Use a subquery


In [176]:
query_executer("select MAX(PAYLOAD_MASS__KG_) from {}".format(table_name),cnx)

,MAX(PAYLOAD_MASS__KG_)
0,15600


In [179]:
query_executer("select Booster_Version from {0} where PAYLOAD_MASS__KG_ = (select MAX(PAYLOAD_MASS__KG_) from {0})".format(table_name),cnx)

,Booster_Version
0,F9 B5 B1048.4
1,F9 B5 B1049.4
2,F9 B5 B1051.3
3,F9 B5 B1056.4
4,F9 B5 B1048.5
5,F9 B5 B1051.4
6,F9 B5 B1049.5
7,F9 B5 B1060.2
8,F9 B5 B1058.3
9,F9 B5 B1051.6


### Task 9

##### List the failed landing_outcomes in drone ship, their booster versions, and launch site names for in year 2015


In [183]:
query_executer("select Landing_Outcome, Booster_Version, Launch_Site, Date from {0} where Landing_Outcome = 'Failure (drone ship)' and Date like '2015%'".format(table_name),cnx)

,Landing_Outcome,Booster_Version,Launch_Site,Date
0,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40,2015-01-10
1,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40,2015-04-14


### Task 10

##### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order


In [202]:
query_executer("""select distinct Landing_Outcome, count(Landing_Outcome) as total from {0} 
where Date>'2010-06-04' and Date<'2017-03-20' group by Landing_Outcome order by total desc""".format(table_name),cnx)

,Landing_Outcome,total
0,No attempt,10
1,Failure (drone ship),5
2,Success (drone ship),5
3,Controlled (ocean),3
4,Success (ground pad),3
5,Uncontrolled (ocean),2
6,Failure (parachute),1
7,Precluded (drone ship),1


### Reference Links

*   <a href ="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20String%20Patterns%20-%20Sorting%20-%20Grouping/instructional-labs.md.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01&origin=www.coursera.org">Hands-on Lab : String Patterns, Sorting and Grouping</a>

*   <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Built-in%20functions%20/Hands-on_Lab__Built-in_Functions.md.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01&origin=www.coursera.org">Hands-on Lab: Built-in functions</a>

*   <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Sub-queries%20and%20Nested%20SELECTs%20/instructional-labs.md.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01&origin=www.coursera.org">Hands-on Lab : Sub-queries and Nested SELECT Statements</a>

*   <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-3-SQLmagic.ipynb?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01">Hands-on Tutorial: Accessing Databases with SQL magic</a>

*   <a href= "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-4-Analyzing.ipynb?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01">Hands-on Lab: Analyzing a real World Data Set</a>


## Author(s)

<h4> Lakshmi Holla </h4>


## Other Contributors

<h4> Rav Ahuja </h4>


## Change log

| Date       | Version | Changed by    | Change Description        |
| ---------- | ------- | ------------- | ------------------------- |
| 2021-10-12 | 0.4     | Lakshmi Holla | Changed markdown          |
| 2021-08-24 | 0.3     | Lakshmi Holla | Added library update      |
| 2021-07-09 | 0.2     | Lakshmi Holla | Changes made in magic sql |
| 2021-05-20 | 0.1     | Lakshmi Holla | Created Initial Version   |


## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
